In [56]:
# imports
import pandas as pd
import numpy as np
import yaml
from yaml.loader import SafeLoader

from python.helpers.pandas_helper import PandasHelper
from python.helpers.bibtex_helper import BibtexHelper
from python.helpers.csv_helper import CSVHelper

bibtex_helper = BibtexHelper
pandas_helper = PandasHelper
csv_helper = CSVHelper

# Aula 1 - Bibtex

In [57]:
with open('config.yaml') as f:
    config = yaml.load(f, Loader=SafeLoader)

df_acm = bibtex_helper.read_bibtex_files('dados_bibtex/ACM')
df_ieee = bibtex_helper.read_bibtex_files('dados_bibtex/IEEE')
df_sdc = bibtex_helper.read_bibtex_files('dados_bibtex/SDC')

df_acm = bibtex_helper.cleaner_columns(df=df_acm)
df_ieee = bibtex_helper.cleaner_columns(df=df_ieee)
df_sdc = bibtex_helper.cleaner_columns(df=df_sdc)

df_bib = pandas_helper.concat_df(df_1=df_acm, df_2=df_ieee)
df_bib = pandas_helper.concat_df(df_1=df_bib, df_2=df_sdc)

df_bib['title'] = df_bib['title'].str.lower()

pandas_helper.save_df(df=df_bib, file_type=config['file_saved_format_bibtex'], file_name_saved=config['name_file_saved_bibtex'])

Arquivos listado no Dataframe: ACM_34.bib
Arquivos listado no Dataframe: ACM_24.bib
Arquivos listado no Dataframe: ACM_13.bib
Arquivos listado no Dataframe: ACM_12.bib
Arquivos listado no Dataframe: ACM_26.bib
Arquivos listado no Dataframe: ACM_3.bib
Arquivos listado no Dataframe: ACM_5.bib
Arquivos listado no Dataframe: ACM_28.bib
Arquivos listado no Dataframe: ACM_18.bib
Arquivos listado no Dataframe: ACM_38.bib
Arquivos listado no Dataframe: ACM_36.bib
Arquivos listado no Dataframe: ACM_4.bib
Arquivos listado no Dataframe: ACM_31.bib
Arquivos listado no Dataframe: ACM_27.bib
Arquivos listado no Dataframe: ACM_29.bib
Arquivos listado no Dataframe: ACM_8.bib
Arquivos listado no Dataframe: ACM_25.bib
Arquivos listado no Dataframe: ACM_14.bib
Arquivos listado no Dataframe: ACM_7.bib
Arquivos listado no Dataframe: ACM_23.bib
Arquivos listado no Dataframe: ACM_1.bib
Arquivos listado no Dataframe: ACM_2.bib
Arquivos listado no Dataframe: ACM_30.bib
Arquivos listado no Dataframe: ACM_35.bib

# Aula 2 - CSV (JCR e SCIMAGOJR)

In [58]:

with open('config.yaml') as f:
    config = yaml.load(f, Loader=SafeLoader)

df_scimagojr = csv_helper.read_csv_to_pandas('dados_rank/scimagojr_2020.csv')
df_jcs = csv_helper.read_csv_to_pandas('dados_rank/jcs_2020.csv')

# Dropa colunas que só possuem NaN
df_scimagojr.dropna(how='all', axis=1, inplace=True)
df_jcs.dropna(how='all', axis=1, inplace=True)

# Passa o nome das colunas pra letra minuscula e checa o nome e altera
csv_helper.cleaner_columns_csv(df=df_scimagojr, column_check='title', column_new_name='title')
csv_helper.cleaner_columns_csv(df=df_jcs, column_check='title', column_new_name='title')

In [59]:
# Passa todos os valores do campo "title" para letra minuscula
df_jcs['title'] = df_jcs['title'].str.lower()
df_scimagojr['title'] = df_scimagojr['title'].str.lower()

In [60]:
df_merged_csv = pandas_helper.merge_dataframes(df_1=df_scimagojr, df_2=df_jcs, column_to_merge='title')

In [61]:
df_merged_csv.head(5)

,rank_x,sourceid,title,type,issn,sjr,sjr_best_quartile,h_index,total_docs__2020,total_docs__3years,...,ref___doc_,country,region,publisher,coverage,categories,rank_y,total_cites,journal_impact_factor,eigenfactor_score
0,1,28773,ca-a cancer journal for clinicians,journal,"15424863, 00079235","62,937",Q1,168,47,119,...,"73,45",United States,Northern America,Wiley-Blackwell,1950-2020,Hematology (Q1); Oncology (Q1),1,"55,868",508.702,0.10514
1,2,19434,mmwr recommendations and reports,journal,"10575987, 15458601","40,949",Q1,143,10,9,...,"129,20",United States,Northern America,Centers for Disease Control and Prevention (CDC),1990-2020,Epidemiology (Q1); Health Information Manageme...,14,"3,288",55.857,0.01123
2,3,20315,nature reviews molecular cell biology,journal,"14710072, 14710080","37,461",Q1,431,115,338,...,"73,38",United Kingdom,Western Europe,Nature Publishing Group,2000-2020,Cell Biology (Q1); Molecular Biology (Q1),2,"58,477",94.444,0.07548
3,4,29431,quarterly journal of economics,journal,"00335533, 15314650","34,573",Q1,259,40,110,...,"68,33",United Kingdom,Western Europe,Oxford University Press,1886-2020,Economics and Econometrics (Q1),215,"38,427",15.563,0.06169
4,5,21100812243,nature reviews materials,journal,20588437,"32,011",Q1,108,92,264,...,"115,57",United Kingdom,Western Europe,Nature Publishing Group,2016-2020,"Biomaterials (Q1); Electronic, Optical and Mag...",7,"19,887",66.308,0.05677


In [62]:
# Muda os nomes de algumas colunas
df_merged_csv.rename(columns={'rank_x': 'scimago_value'}, inplace=True)
df_merged_csv.rename(columns={'rank_y': 'jcr_value'}, inplace=True)

In [63]:
# separa a coluna issn por ", "
df_merged_csv['issn'] = df_merged_csv['issn'].map(lambda x: x.split(', ') if x==x else np.nan)
df_merged_csv['issn'].head(10)

0    [15424863, 00079235]
1    [10575987, 15458601]
2    [14710072, 14710080]
3    [00335533, 15314650]
4              [20588437]
5    [00928674, 10974172]
6    [14710056, 14710064]
7    [00346861, 15390756]
8    [00223808, 1537534X]
9    [14741741, 14741733]
Name: issn, dtype: object

In [64]:
LENGHT = []
for value in df_merged_csv['issn'].values:
    if value is not np.nan:
        LENGHT.append(len(value))

MAX_LEN = max(LENGHT) 
print(MAX_LEN)

3


In [65]:
for i in range(MAX_LEN):
    if i == 0:
        df_merged_csv[f'issn_{i}'] = df_merged_csv['issn'].map(lambda x: x[i] if x == x else np.nan)
    else:
        df_merged_csv[f'issn_{i}'] = df_merged_csv['issn'].map(lambda x: np.nan if x !=x else x[i] if len(x) > i else x[0])

df_merged_csv.drop('issn', axis = 1, inplace = True)
df_merged_csv[['issn_0', 'issn_1','issn_2']]

,issn_0,issn_1,issn_2
0,15424863,00079235,15424863
1,10575987,15458601,10575987
2,14710072,14710080,14710072
3,00335533,15314650,00335533
4,20588437,20588437,20588437
...,...,...,...
10423,2193651X,21938237,2193651X
10424,26624729,26624737,26624729
10425,18517471,03735680,18517471
10426,25424653,25424653,25424653


In [66]:
df_bib['issn'] = df_bib['issn'].map(lambda x: x.replace('-','') if x == x else np.nan)

In [67]:
df_issn_main = pd.DataFrame()
for i in range(MAX_LEN):
    df_merged_csv_bib_issn = pd.DataFrame()

    df_merged_csv_bib_issn = df_merged_csv.merge(df_bib, how = 'outer', left_on=f'issn_{i}', right_on = 'issn', copy=False, indicator = True)
    df_merged_csv_bib_issn = df_merged_csv_bib_issn.loc[(df_merged_csv_bib_issn._merge == 'both') & (df_merged_csv_bib_issn.issn.notna())]
    df_merged_csv_bib_issn.drop('_merge', axis = 1, inplace = True)

    df_issn_main = pandas_helper.concat_df(df_1=df_issn_main, df_2=df_merged_csv_bib_issn)

In [68]:
df_issn_main = pandas_helper.remove_duplicate(df=df_issn_main)


In [69]:
# Muda os nomes de algumas colunas
df_issn_main.rename(columns={'title_x': 'title_csv'}, inplace=True)
df_issn_main.rename(columns={'title_y': 'title_bib'}, inplace=True)

In [70]:
df_issn_main.head(10)

,scimago_value,sourceid,title_csv,type,sjr,sjr_best_quartile,h_index,total_docs__2020,total_docs__3years,total_refs_,...,type_publication,id,number,volume,journal,booktitle,issn,note,editor,edition
250,289.0,2.114900e+04,leadership quarterly,journal,"4,989",Q1,151.0,78.0,150.0,9416.0,...,article,LEE2020101426,NaN,NaN,The Leadership Quarterly,NaN,10489843,NaN,NaN,NaN
283,326.0,1.760016e+10,molecular plant,journal,"4,588",Q1,115.0,153.0,474.0,8872.0,...,article,XU2022,NaN,NaN,Molecular Plant,NaN,16742052,NaN,NaN,NaN
284,326.0,1.760016e+10,molecular plant,journal,"4,588",Q1,115.0,153.0,474.0,8872.0,...,article,UEDA2019150,2,12,Molecular Plant,NaN,16742052,NaN,NaN,NaN
285,327.0,1.780900e+04,trends in plant science,journal,"4,587",Q1,263.0,162.0,386.0,9428.0,...,article,SUN2022191,2,27,Trends in Plant Science,NaN,13601385,NaN,NaN,NaN
384,432.0,2.897300e+04,journal of econometrics,journal,"3,769",Q1,159.0,241.0,418.0,11045.0,...,article,ANDREASEN201926,1,212,Journal of Econometrics,NaN,03044076,Big Data in Dynamic Predictive Econometric Mod...,NaN,NaN
444,497.0,1.300500e+05,ocular surface,journal,"3,505",Q1,65.0,120.0,234.0,5895.0,...,article,PURANIK2019838,4,17,The Ocular Surface,NaN,15420124,NaN,NaN,NaN
490,551.0,1.654700e+04,tourism management,journal,"3,328",Q1,199.0,159.0,690.0,11910.0,...,article,LI2018301,NaN,68,Tourism Management,NaN,02615177,NaN,NaN,NaN
518,581.0,1.614600e+04,trends in biotechnology,journal,"3,192",Q1,219.0,167.0,397.0,11206.0,...,article,GOH20221029,9,40,Trends in Biotechnology,NaN,01677799,NaN,NaN,NaN
521,585.0,2.299000e+04,journal of retailing,journal,"3,184",Q1,136.0,66.0,98.0,4311.0,...,article,BRADLOW201779,1,93,Journal of Retailing,NaN,00224359,The Future of Retailing,NaN,NaN
533,599.0,1.239600e+04,journal of strategic information systems,journal,"3,133",Q1,88.0,24.0,75.0,2111.0,...,article,JONES20193,1,28,The Journal of Strategic Information Systems,NaN,09638687,NaN,NaN,NaN


In [71]:
df_filter = pandas_helper.filter_column(df=df_issn_main, column_filter=config['columns_filters'])

In [72]:
df_filter.head(10)

,scimago_value,sourceid,title_csv,type,sjr,sjr_best_quartile,h_index,total_docs__2020,total_docs__3years,total_refs_,...,type_publication,id,number,volume,journal,booktitle,issn,note,editor,edition
1486,1603.0,1.953200e+04,accident analysis and prevention,journal,"1,816",Q1,152.0,385.0,1041.0,20484.0,...,article,ESKANDARITORBAGHAN2022106543,NaN,166,Accident Analysis & Prevention,NaN,00014575,NaN,NaN,NaN
711,769.0,1.326600e+04,american journal of ophthalmology,journal,"2,704",Q1,186.0,428.0,1146.0,11979.0,...,article,COLEMAN20151091,6,160,American Journal of Ophthalmology,NaN,00029394,NaN,NaN,NaN
283,326.0,1.760016e+10,molecular plant,journal,"4,588",Q1,115.0,153.0,474.0,8872.0,...,article,XU2022,NaN,NaN,Molecular Plant,NaN,16742052,NaN,NaN,NaN
668,736.0,1.563100e+04,international journal of information management,journal,"2,770",Q1,114.0,224.0,382.0,20318.0,...,article,LARSON2016700,5,36,International Journal of Information Management,NaN,02684012,NaN,NaN,NaN
785,852.0,2.185800e+04,british journal of anaesthesia,journal,"2,589",Q1,181.0,486.0,1260.0,12638.0,...,article,MCISAAC2020510,5,124,British Journal of Anaesthesia,NaN,00070912,NaN,NaN,NaN
937,1044.0,1.496600e+04,journal of manufacturing systems,journal,"2,310",Q1,70.0,155.0,294.0,8906.0,...,article,PENG2022104,NaN,65,Journal of Manufacturing Systems,NaN,02786125,NaN,NaN,NaN
1081,1183.0,2.010700e+04,survey of ophthalmology,journal,"2,131",Q1,132.0,69.0,235.0,6991.0,...,article,CLARK2016443,4,61,Survey of Ophthalmology,NaN,00396257,NaN,NaN,NaN
1571,1673.0,1.695600e+04,cities,journal,"1,771",Q1,90.0,419.0,732.0,28409.0,...,article,LV2021103298,NaN,116,Cities,NaN,02642751,NaN,NaN,NaN
3024,3401.0,1.233200e+04,safety science,journal,"1,178",Q1,111.0,451.0,1047.0,26276.0,...,article,HUANG201846,NaN,109,Safety Science,NaN,09257535,NaN,NaN,NaN
3025,3401.0,1.233200e+04,safety science,journal,"1,178",Q1,111.0,451.0,1047.0,26276.0,...,article,OUYANG201860,NaN,101,Safety Science,NaN,09257535,NaN,NaN,NaN


# Salva o arquivo

In [73]:
pandas_helper.save_df(df=df_filter, file_type=config['file_saved_format_rank'], file_name_saved=config['name_file_saved_rank'])

Arquivo salvo: resultados/results_rank.yaml
